# Interactive plate heatmap viewer with image tooltips demonstration

Volker Hilsenstein<p> 
EMBL Advanced Light Microscopy Facility, Heidelberg<p>
April 2017

## Aim

For high-throughput screening experiments in plate formats, one often wants to visually inspect heat maps showing the whole plate to detect systematic errors like gradients across the plate and to detect wells with hits (strong effects). 

However, the numbers that make up the heatmap represent measurements extracted with image analysis. If a well appears to be a hit one would often like to check wether the image analysis (or the microscopy) worked correctly. 
Therefore it is important to be able to link back from the points in the plot to the images.

This heatmap viewer implements the functionality to visualize several plates in a high-throughput screen and link back to image sequences.

## Implementation

The implementation is with __holoviews__ using the __bokeh__ backend.
As I can't make the screening data of our users publicly available, the first part of this notebook simply 
generates a data frame with random data.
To demonstrate the ability to link back to images, I created a list of animated gif URLs based on a search for "microscopy" on Giphy. See the notebook `GetAnimatedGifURLs.ipynb`. 


## Requirements

I dumped my python environmnent in a .yml file `plateviewer_conda_env.yml`. However, this environment has many more packages and channels that you will need. Here is a manually edited version that includes the `holoviews`, `bokeh` and `pandas` versions that I used.

```
name: base
channels:
  - pyviz
  - pytorch
  - damianavila82
  - bokeh
  - ioam
  - bioconda
  - conda-forge
  - r
  - default
  - conda
  - defaults
dependencies:
  - bokeh=0.12.14=py36_1
  - conda=4.5.1=py36_0
  - conda-env=2.6.0=0
  - dask=0.17.0=py_0
  - dask-core=0.17.0=py_0
  - datashader=0.6.5=py_0
  - holoviews=1.9.5=py36_0
  - ipywidgets=7.0.1=py_2
  - jupyter_client=5.2.3=py36_0
  - jupyter_core=4.4.0=py_0
  - matplotlib=2.1.1=py36_0
  - notebook=5.4.1=py36_0
  - pandas=0.22.0=py36_0
  - tornado=4.5.3=py36_0
  - widgetsnbextension=3.0.3=py36_2
  - jupyter=1.0.0=py36_3
  - numba=0.36.2=np112py36h37f72b1_0
  - numpy=1.12.1=py36h8871d66_1
  - numpydoc=0.6.0=py36_0
  - pip=9.0.1=py36_1
  - python=3.6.0=0
  - yaml=0.1.6=0
  - zlib=1.2.8=3
  - param=1.5.0=py36_0
  - parambokeh=0.2.2=py_0
  - pip:
    - imagen==2.1.0
    - imutils==0.4.5
    - jupyterthemes==0.16.4
    - nbdime==0.4.1
    - paho-mqtt==1.3.1
    - pandas-datareader==0.5.0

```


# Import required packages

In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
hv.extension('bokeh')
import pandas as pd
import re
from plateviewer_tools import well_nr_to_coords, create_custom_tooltip

# Create synthetic data

In [ ]:
def createDF():
    df = pd.DataFrame(np.random.randn(96, 4), columns=list('ABCD'))
    df["well"] = np.arange(96)+1
    # change the scaling
    df["A"] *= 255
    df["B"] *= 10e3
    df["C"] *= 30e6
    return df



In [ ]:
df1 = createDF()
df1 = pd.melt(df1, id_vars="well", var_name="Platename", value_name="value1")
df2 = createDF()
df2 = pd.melt(df2, id_vars="well", var_name="Platename", value_name="value2")


## Let's also create some outliers in the value2 column.
This demonstrates the need for having the option to interactively scale the color map

In [ ]:
df = pd.concat([df1, df2.value2], axis=1)
# create some outliers in a few plates

df.loc[df.well==20, "value2"] *= 30
#df.loc[df.well==23, "value2"] = np.nan

## Add well coordinate columns based on well numbers

In [ ]:
wellcoords = df.well.apply(well_nr_to_coords)
df = pd.concat([df, wellcoords],axis=1)

# Add image URLs

In [ ]:
gifs = pd.read_csv("giphy_urls.csv")
df = pd.concat([df, gifs[0:len(df)].giffile], axis=1)
# note ! not all URLs are unique. Expect to see the same gif in several wells !

## In case the data isn't sorted already you would have to sort by row and column, otherwise the heatmap is in random order

In [ ]:
df  = df.sort_values(['row','col'], ascending=[False,True])


## This is what the final data frame looks like

In [ ]:
df.head()

# Generate the custom tooltip html for the Hover Tool

In [ ]:
value_fields = ["value1", "value2", "well"]
info_fields = [ "well", "value1", "value2", "Platename"]

impath = ""
im_col_name = "giffile"

custom_tip = create_custom_tooltip(info_fields, impath, im_col_name, 100, 100)
#custom_tip = ""
from bokeh.models import HoverTool
hover = HoverTool( tooltips= custom_tip)

In [ ]:
print(custom_tip)

## Now setup the dynamic map for interactive heatmap visualization

In [ ]:
width = 700
height = int(width * 8/12)


# from 
# https://stackoverflow.com/questions/46024901/how-to-format-colorbar-label-in-bokeh
from bokeh.models import PrintfTickFormatter
formatter = PrintfTickFormatter(format='%1f')
colorbar_opts={'formatter': formatter}


def select_plate(Platename, Column, cmap, lower_percentile, upper_percentile):
    #print(f"Platename: {Platename}, Column: {Column}, Normalisation: {Normalisation}")
    tmp = df[df["Platename"]==Platename]
    values = tmp[Column]
    vdims = [Column] + list(set(info_fields + ["giffile"]) - set([Column]))
    kdims = ['col','row']
    cmap_range = (np.percentile(values, lower_percentile ), np.percentile(values, upper_percentile))             
    dataset = hv.Dataset(tmp, vdims=vdims, kdims=kdims)
    heatmap = hv.HeatMap(dataset, vdims=vdims, kdims=kdims).redim.range(**{Column: cmap_range}).options(tools=[hover], logz=False ,colorbar=True ,width=width, height=height, toolbar='below', colorbar_opts=colorbar_opts, invert_yaxis=True, cmap=cmap)
    return  heatmap
 
    
hv.DynamicMap(select_plate, kdims=['Platename','Column', 'cmap', 'lower_percentile','upper_percentile', ]).redim.values(Platename=pd.unique(df["Platename"]), Column=["value1", "value2"], cmap=["bwr", "coolwarm"], lower_percentile=list(range(50)),upper_percentile=list(range(50,101)))